# 01 - Data Analysis and Preparation

This notebook covers the following tasks:

1. Perform `Exploratory Data Analysis` and `Visualization`.
2. Prepare the data for the ML task in `BigQuery`.
3. Generate and fix the raw data schema.
4. Create a `Vertex Dataset` resource (i.e., managed dataset).


## Dataset

The [Chicago Taxi Trips](https://pantheon.corp.google.com/marketplace/details/city-of-chicago-public-data/chicago-taxi-trips) dataset is one of [public datasets hosted with BigQuery](https://cloud.google.com/bigquery/public-data/), which includes taxi trips from 2013 to the present, reported to the City of Chicago in its role as a regulatory agency. The `taxi_trips` table size is 70.72 GB and includes more than 195 million records. The dataset includes information about the trips, like pickup and dropoff datetime and location, passengers count, miles travelled, and trip toll. 

The ML task is to predict whether a given trip will result in a tip > 20%.

## Setup

Builtin Jupyter command to reload all Python imported modules.

## Installation

Install the latest version of Vertex SDK.

In [ ]:
import sys
import os


# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = '--user'
else:
    USER_FLAG = ''

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG

Install the latest GA version of *google-cloud-storage* library as well.

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

Install deep learning dependencies

In [ ]:
! pip3 install -U tfx==0.30.0 $USER_FLAG
! pip3 install -r requirements.txt $USER_FLAG
! pip3 install --upgrade google-cloud-bigquery-storage $USER_FLAG
! pip3 install --upgrade google-cloud-bigquery $USER_FLAG

### Restart the kernel

Once you've installed the Vertex SDK and Google *cloud-storage*, you need to restart the notebook kernel so it can find the packages.

In [ ]:
if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

In [ ]:
%load_ext autoreload
%autoreload 2?

In [ ]:
import os
import pandas as pd
import tensorflow as tf
import tensorflow_data_validation as tfdv
from google.cloud import bigquery
import matplotlib.pyplot as plt


from google.cloud import aiplatform as vertex_ai
from google.cloud import aiplatform_v1beta1 as vertex_ai_beta

### Setup your Google Cloud project

Enter your project ID in the cell below. Then run the  cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

In [ ]:
PROJECT_ID = "[your-project-id]"  #@param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

! gcloud config set project $PROJECT_ID

#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You may not use a multi-regional bucket for training with Vertex. Not all regions provide support for all Vertex services. For the latest support per region, see the [Vertex locations documentation](https://cloud.google.com/ai-platform-unified/docs/general/locations)

In [ ]:
REGION = 'us-central1'  #@param {type: "string"}

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

In [ ]:
BQ_DATASET_NAME = 'playground_us' # Change to your BQ datasent name.
BQ_TABLE_NAME = 'chicago_taxitrips_prep'
BQ_LOCATION = 'US'

RAW_SCHEMA_DIR = 'src/raw_schema'

DATASET_DISPLAY_NAME = 'chicago_taxi_tips'
BQ_URI = f"bq://{PROJECT_ID}.{BQ_DATASET_NAME}.{BQ_TABLE_NAME}"

In [ ]:
!bq --location=US mk -d \
$PROJECT_ID:$BQ_DATASET_NAME

## 1. Explore the data in BigQuery

In [ ]:
sql_script = '''
SELECT 
    CAST(EXTRACT(DAYOFWEEK FROM trip_start_timestamp) AS string) AS trip_dayofweek, 
    FORMAT_DATE('%A',cast(trip_start_timestamp as date)) AS trip_dayname,
    COUNT(*) as trip_count,
FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
WHERE
    EXTRACT(YEAR FROM trip_start_timestamp) = 2015 
GROUP BY
    trip_dayofweek,
    trip_dayname
ORDER BY
    trip_dayofweek
;
'''

In [ ]:
bq_client = bigquery.Client(project=PROJECT_ID, location=BQ_LOCATION)
job = bq_client.query(sql_script)

In [ ]:
result = job.result()
data=[]
for row in result:
    data.append(row.values())
    
from pandas import DataFrame
df = DataFrame(data, columns=['id', 'trip_dayname', 'trip_count'])

Display data item counts per day of the week in table format

In [ ]:
df

Display data item counts per day of the week as a bar graph

In [ ]:
df.plot(kind='bar', x='trip_dayname', y='trip_count')

## 2. Create data for the ML task

We add a `ML_use` column for pre-splitting the data, where 80% of the datsa items are set to `UNASSIGNED` while the other 20% is set to `TEST`.

This column is used during training (custom and AutoML) to split the dataset for training and test.

In the training phase, the `UNASSIGNED` are split into `train` and `eval`. The `TEST` split is will be used for the final model validation.

In [ ]:
# Use maximum of 1M data items
sample_size = 1000000

# Limit data items to the year 2020
year = 2020

In [ ]:
sql_script = '''
CREATE OR REPLACE TABLE `@PROJECT_ID.@DATASET.@TABLE` 
AS (
    WITH
      taxitrips AS (
      SELECT
        trip_start_timestamp,
        trip_seconds,
        trip_miles,
        payment_type,
        pickup_longitude,
        pickup_latitude,
        dropoff_longitude,
        dropoff_latitude,
        tips,
        fare
      FROM
        `bigquery-public-data.chicago_taxi_trips.taxi_trips`
      WHERE 1=1 
      AND pickup_longitude IS NOT NULL
      AND pickup_latitude IS NOT NULL
      AND dropoff_longitude IS NOT NULL
      AND dropoff_latitude IS NOT NULL
      AND trip_miles > 0
      AND trip_seconds > 0
      AND fare > 0
      AND EXTRACT(YEAR FROM trip_start_timestamp) = @YEAR
    )

    SELECT
      trip_start_timestamp,
      EXTRACT(MONTH from trip_start_timestamp) as trip_month,
      EXTRACT(DAY from trip_start_timestamp) as trip_day,
      EXTRACT(DAYOFWEEK from trip_start_timestamp) as trip_day_of_week,
      EXTRACT(HOUR from trip_start_timestamp) as trip_hour,
      trip_seconds,
      trip_miles,
      payment_type,
      ST_AsText(
          ST_SnapToGrid(ST_GeogPoint(pickup_longitude, pickup_latitude), 0.1)
      ) AS pickup_grid,
      ST_AsText(
          ST_SnapToGrid(ST_GeogPoint(dropoff_longitude, dropoff_latitude), 0.1)
      ) AS dropoff_grid,
      ST_Distance(
          ST_GeogPoint(pickup_longitude, pickup_latitude), 
          ST_GeogPoint(dropoff_longitude, dropoff_latitude)
      ) AS euclidean,
      CONCAT(
          ST_AsText(ST_SnapToGrid(ST_GeogPoint(pickup_longitude,
              pickup_latitude), 0.1)), 
          ST_AsText(ST_SnapToGrid(ST_GeogPoint(dropoff_longitude,
              dropoff_latitude), 0.1))
      ) AS loc_cross,
      IF((tips/fare >= 0.2), 1, 0) AS tip_bin,
      IF(RAND() <= 0.8, 'UNASSIGNED', 'TEST') AS ML_use
    FROM
      taxitrips
    LIMIT @LIMIT
)
'''

In [ ]:
sql_script = sql_script.replace(
    '@PROJECT_ID', PROJECT_ID).replace(
    '@DATASET', BQ_DATASET_NAME).replace(
    '@TABLE', BQ_TABLE_NAME).replace(
    '@YEAR', str(year)).replace(
    '@LIMIT', str(sample_size))

In [ ]:
print(sql_script)

In [ ]:
bq_client = bigquery.Client(project=PROJECT_ID, location=BQ_LOCATION)
job = bq_client.query(sql_script)
job.result()

In [ ]:
sql_script = '''
SELECT ML_use, COUNT(*)
FROM PROJECT_ID.playground_us.chicago_taxitrips_prep
GROUP BY ML_use
'''.replace('PROJECT_ID', PROJECT_ID)

print(sql_script)

In [ ]:
job = bq_client.query(sql_script)
result = job.result()
for row in result:
    print(row)

### Load a sample data to a Pandas dataframe

In [ ]:
sql_script = '''
SELECT * EXCEPT (trip_start_timestamp, ML_use)
FROM PROJECT_ID.playground_us.chicago_taxitrips_prep
'''.replace('PROJECT_ID', PROJECT_ID)

print(sql_script)

In [ ]:
job = bq_client.query(sql_script)
sample_data = job.result().to_dataframe()

In [ ]:
sample_data.head().T

In [ ]:
sample_data.tip_bin.value_counts()

In [ ]:
sample_data.euclidean.hist()

## 3. Generate the raw data schema

The raw data schema will be used in:
1. Defining the input columns for the `AutoML Tabular` model.
2. Indentifying the raw data types and shapes in the feature transformations.
3. Create the serving input signature for the custom model.
4. Validating the new raw training data in the TFX pipeline.

In [ ]:
stats = tfdv.generate_statistics_from_dataframe(
    dataframe=sample_data,
    stats_options=tfdv.StatsOptions(
        label_feature='tip_bin',
        weight_feature=None,
        sample_rate=1,
        num_top_values=50
    )
)

In [ ]:
tfdv.visualize_statistics(stats)

In [ ]:
schema = tfdv.infer_schema(statistics=stats)
tfdv.display_schema(schema=schema)

In [ ]:
raw_schema_location = os.path.join(RAW_SCHEMA_DIR, 'schema.pbtxt')
tfdv.write_schema_text(schema, raw_schema_location)

## 4. Create `Vertex Dataset` resource

### Create `Dataset` resource

In [ ]:
dataset = vertex_ai.TabularDataset.create(
            display_name=DATASET_DISPLAY_NAME, bq_source=BQ_URI)

print(dataset.gca_resource)

### Get `Dataset` resource

In [ ]:
datasets = vertex_ai.TabularDataset.list(filter=f"display_name={DATASET_DISPLAY_NAME}")
dataset = datasets[0]
dataset.resource_name 

In [ ]:
dataset.gca_resource.metadata['inputConfig']['bigquerySource']['uri']